## 1

A random sample of 10 persons had the following IQs.:
* ```70,120,110,101,88,83,95,98,107,100```

Does the data support the assumption of a population mean of IQ of 100 (at 5 % level of significance)

(Ans. Yes:

* p value: 0.550419 (greater than 0.05)
* t-test statistic value: -0.620,  critical value: +- 2.262)

In [ ]:
import numpy as np
from scipy import stats

In [ ]:
# IQ data
data = [70, 120, 110, 101, 88, 83, 95, 98, 107, 100]

# Compute statistics
mean = np.mean(data)
std = np.std(data, ddof=1)
n = len(data)
df = n - 1

In [ ]:
# Manual t-test
t_statistic = (mean - 100) / (std / np.sqrt(n))
p_value = 2 * stats.t.cdf(-abs(t_statistic), df)
critical_value = stats.t.ppf(0.975, df)

In [ ]:
print(f"Sample mean: {mean:.2f}")
print(f"Sample standard deviation: {std:.2f}")
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.6f}")
print(f"Critical value (alpha=0.05): ±{critical_value:.3f}")

# Alternatively, using SciPy's built-in function
t_scipy, p_scipy = stats.ttest_1samp(data, 100)
print(f"\nSciPy t-test → t = {t_scipy:.3f}, p = {p_scipy:.6f}")

Sample mean: 97.20
Sample standard deviation: 14.27
t-statistic: -0.620
p-value: 0.550419
Critical value (alpha=0.05): ±2.262

SciPy t-test → t = -0.620, p = 0.550419


The p-value (0.5504) is greater than 0.05, and the t-statistic (–0.620) lies between the critical values ±2.262, so we fail to reject the hypothesis that the population mean IQ is 100.



## 2

Engine parts with the axle diameter of 0.7 cm are manufactured. A random sample of produced parts axle diameters are in the file ```sample3.csv```. Based on the sample, would you say that the work is inferior? (Is the sample mean significantly different from the stated value ot 0.7 cm? Use the significance level 0.05)

(Ans. It is inferior:
* p value: 0.002602
* t-test statistic value: 4.119 and critical value: +- 2.262)

In [2]:
# 1. Upload the file
from google.colab import files
uploaded = files.upload()  # select sample3.csv when prompted

# 2. Read into DataFrame
import io
import pandas as pd
import scipy.stats as stats

fname = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[fname]))

# 3. Extract the sample column
#    (assumes your CSV has only one column of diameters)
sample = df.iloc[:, 0]

# 4. Given parameters
mu = 0.7
alpha = 0.05

# 5. Sample statistics
n = len(sample)
mean = sample.mean()
std = sample.std(ddof=1)
t_stat = (mean - mu) / (std / (n**0.5))
dfree = n - 1
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), dfree))
crit_value = stats.t.ppf(1 - alpha/2, dfree)

# 6. Display results
print(f"Sample size (n): {n}")
print(f"Sample mean: {mean:.5f}")
print(f"Sample std dev: {std:.5f}")
print(f"t-statistic: {t_stat:.3f}")
print(f"Degrees of freedom: {dfree}")
print(f"Critical t-value (±): {crit_value:.3f}")
print(f"p-value: {p_value:.6f}")


Saving sample3.csv to sample3.csv
Sample size (n): 9
Sample mean: 0.74548
Sample std dev: 0.03772
t-statistic: 3.618
Degrees of freedom: 8
Critical t-value (±): 2.306
p-value: 0.006803


The sample’s average axle diameter is approximately **0.745 cm** instead of the target **0.700 cm**.  
Our test statistic (t = 3.618) exceeds the critical value (±2.306), and the p-value (0.0068) is below 0.05.  
Therefore, we reject the null hypothesis and conclude that the manufactured parts are significantly off-spec and the process is inferior.


# 3
Repeat part 1 using the bootstrap method

In [1]:
import numpy as np

# Original data
data = np.array([70, 120, 110, 101, 88, 83, 95, 98, 107, 100])

# Number of bootstrap samples
n_boot = 10000
np.random.seed(42)

# Bootstrap distribution of the sample mean
boot_means = np.array([np.mean(np.random.choice(data, size=len(data), replace=True))
                       for _ in range(n_boot)])
ci_lower, ci_upper = np.percentile(boot_means, [2.5, 97.5])

# Bootstrap p-value for testing H0: mean = 100
observed_mean = np.mean(data)
# Center data under null hypothesis
data_centered = data - observed_mean + 100
boot_means_null = np.array([
    np.mean(np.random.choice(data_centered, size=len(data_centered), replace=True))
    for _ in range(n_boot)
])
# Two-sided p-value
p_boot = np.mean(np.abs(boot_means_null - 100) >= abs(observed_mean - 100))

# Display results
print(f"Observed sample mean: {observed_mean:.3f}")
print(f"Bootstrap 95% CI for mean: [{ci_lower:.3f}, {ci_upper:.3f}]")
print(f"Bootstrap p-value: {p_boot:.4f}")


Observed sample mean: 97.200
Bootstrap 95% CI for mean: [88.500, 105.200]
Bootstrap p-value: 0.5188


The 95 % bootstrap confidence interval covers 100, and the bootstrap p-value (0.5188) is much larger than 0.05.
Yes, the data are consistent with a population mean IQ of 100.

# 4
Repeat part 2 using bootstrap

In [3]:
import numpy as np
import pandas as pd

# 1. Load your data
df = pd.read_csv('sample3.csv')       # or adjust path if needed
sample = df.iloc[:, 0].values         # assumes a single column of diameters

# 2. Observed statistics
mu = 0.7
obs_mean = sample.mean()

# 3. Create a “shifted” sample so that H0 is true (mean = 0.7)
shifted = sample - obs_mean + mu

# 4. Bootstrap the null distribution of the mean
n_boot = 10000
boot_means_null = np.empty(n_boot)
for i in range(n_boot):
    bs = np.random.choice(shifted, size=sample.size, replace=True)
    boot_means_null[i] = bs.mean()

# 5. Two‐sided p‐value
p_value = np.mean(np.abs(boot_means_null - mu) >= np.abs(obs_mean - mu))

# 6. Bootstrap CI for the actual sample mean
boot_means = np.empty(n_boot)
for i in range(n_boot):
    bs = np.random.choice(sample, size=sample.size, replace=True)
    boot_means[i] = bs.mean()
ci_lower, ci_upper = np.percentile(boot_means, [2.5, 97.5])

# 7. Print results
print(f"Observed mean:        {obs_mean:.5f}")
print(f"Bootstrap p-value:    {p_value:.4f}")
print(f"95% bootstrap CI:     [{ci_lower:.5f}, {ci_upper:.5f}]")


Observed mean:        0.74548
Bootstrap p-value:    0.0005
95% bootstrap CI:     [0.72040, 0.76638]


Because the p-value is well below 0.05, we again reject \(H_0\). The parts’ average diameter is significantly different from 0.700 cm—confirming the process is off specification.